In [1]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

2023-10-20 15:25:55.608759: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 15:25:56.194750: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-20 15:25:56.820555: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-20 15:25:56.845101: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [3]:
tf.keras.backend.clear_session()

In [4]:
subject = 'Mato Grosso Do Sul - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [5]:
data = pd.read_csv('2003_mo_model_input_MS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso Do Sul - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.741626,53.166560,8.248824,0.282837,4.562945e+07,2.151917e+06,16.203179,3.957545e+07,33.744
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.741710,53.245714,8.242935,0.284487,4.568706e+07,2.153542e+06,16.206131,3.959155e+07,32.230
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.741794,52.994640,8.237046,0.285379,4.574467e+07,2.155168e+06,16.209084,3.960765e+07,33.559
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.741878,53.019283,8.231157,0.286089,4.580228e+07,2.156794e+06,16.212036,3.962375e+07,34.283
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.741962,53.539593,8.225268,0.286122,4.585990e+07,2.158419e+06,16.214989,3.963985e+07,36.066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.097125,NaN,0.523445,NaN,NaN,NaN,NaN,103.854
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.657488,NaN,0.521590,NaN,NaN,NaN,NaN,95.890
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.966263,NaN,0.518616,NaN,NaN,NaN,NaN,89.679
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.205635,NaN,0.515040,NaN,NaN,NaN,NaN,99.356


In [6]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.739598,-1.432541,1.216541,-0.459341,-1.637645,-2.402906,-1.784438,-1.738619
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.720624,-1.423955,1.206220,-0.422027,-1.622679,-2.361122,-1.774070,-1.724323
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.701651,-1.451188,1.195898,-0.401858,-1.607713,-2.319339,-1.763703,-1.710027
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.682677,-1.448515,1.185577,-0.385815,-1.592747,-2.277555,-1.753336,-1.695732
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.663704,-1.392079,1.175256,-0.385067,-1.577781,-2.235771,-1.742968,-1.681436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.190288,-0.831849,-1.205180,0.316446,1.171465,-0.292680,0.923349,1.021439
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.165389,-0.799920,-1.216425,0.383902,1.161163,-0.292842,0.905256,1.007593
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.140489,-0.711338,-1.227670,0.452548,1.150861,-0.293004,0.887162,0.993747
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.115590,-0.636591,-1.238915,0.528940,1.140559,-0.293166,0.869069,0.979902


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      36.256
1      33.675
2      43.734
3      36.187
4      33.413
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Mato Grosso Do Sul - Consumo de Cimento (t), Length: 240, dtype: float64

In [8]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.739598,-1.432541,1.216541,-0.459341,-1.637645,-2.402906,-1.784438,-1.738619
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.720624,-1.423955,1.206220,-0.422027,-1.622679,-2.361122,-1.774070,-1.724323
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.701651,-1.451188,1.195898,-0.401858,-1.607713,-2.319339,-1.763703,-1.710027
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.682677,-1.448515,1.185577,-0.385815,-1.592747,-2.277555,-1.753336,-1.695732
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.663704,-1.392079,1.175256,-0.385067,-1.577781,-2.235771,-1.742968,-1.681436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.523477,-0.303048,-0.435460,0.586449,1.207622,0.129567,1.062247,1.123802
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.520281,-0.384401,-0.482211,0.549251,1.211468,0.098041,1.062771,1.125391
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.517085,-0.431034,-0.528962,0.512196,1.215313,0.066515,1.063296,1.126980
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.513890,-0.493901,-0.575713,0.470611,1.219159,0.034989,1.063820,1.128569


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      36.256
1      33.675
2      43.734
3      36.187
4      33.413
        ...  
157    66.462
158    79.514
159    63.281
160    77.205
161    71.168
Name: Mato Grosso Do Sul - Consumo de Cimento (t), Length: 162, dtype: float64

In [10]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [11]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [12]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.553261,1.201269,-1.145074,1.284598,0.792487,0.828557,0.883359,0.868222
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.566483,1.181208,-1.093669,1.294211,0.812167,0.824916,0.896439,0.883314
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.579705,1.148281,-1.042264,1.299852,0.831848,0.821275,0.909518,0.898405
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.592927,1.106818,-0.990860,1.306641,0.851528,0.817634,0.922598,0.913496
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.606148,1.062051,-0.939455,1.314465,0.871209,0.813993,0.935678,0.928588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.190288,-0.831849,-1.205180,0.316446,1.171465,-0.292680,0.923349,1.021439
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.165389,-0.799920,-1.216425,0.383902,1.161163,-0.292842,0.905256,1.007593
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.140489,-0.711338,-1.227670,0.452548,1.150861,-0.293004,0.887162,0.993747
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.115590,-0.636591,-1.238915,0.528940,1.140559,-0.293166,0.869069,0.979902


In [13]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [14]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [15]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2494586607, 3835262765, 2539828616, 2343562887, 2450786130, 1668707592, 1689059642, 260798326, 309704583, 2463455595, 315843504, 1222169566, 3215749218, 285295995, 4205351160, 419286863, 3611736698, 1994990944, 567074677, 2497488285]


Step: 0 ___________________________________________


2023-10-20 15:25:57.216795: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-20 15:25:57.217031: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-20 15:25:57.217203: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

val_loss: 25.338098526000977
winner_seed: 2494586607


Step: 1 ___________________________________________
val_loss: 21.11327362060547
winner_seed: 3835262765


Step: 2 ___________________________________________
val_loss: 23.106794357299805


Step: 3 ___________________________________________
val_loss: 28.19892120361328


Step: 4 ___________________________________________
val_loss: 24.030275344848633


Step: 5 ___________________________________________
val_loss: 26.138927459716797


Step: 6 ___________________________________________
val_loss: 18.676210403442383
winner_seed: 1689059642


Step: 7 ___________________________________________
val_loss: 22.811695098876953


Step: 8 ___________________________________________
val_loss: 26.659788131713867


Step: 9 ___________________________________________
val_loss: 21.10254669189453


Step: 10 ___________________________________________
val_loss: 22.553691864013672


Step: 11 ___________________________________________
val_loss: 24.412

In [18]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 131ms/step - loss: 5963.3667 - val_loss: 608.5649
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 749.0027 - val_loss: 654.6655
Epoch 3/10000
4/4 [==============================] - 0s 11ms/step - loss: 649.6091 - val_loss: 814.9406
Epoch 4/10000
4/4 [==============================] - 0s 11ms/step - loss: 798.4180 - val_loss: 1123.1333
Epoch 5/10000
4/4 [==============================] - 0s 11ms/step - loss: 886.2320 - val_loss: 444.1277
Epoch 6/10000
4/4 [==============================] - 0s 12ms/step - loss: 665.1977 - val_loss: 362.0726
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 375.7596 - val_loss: 360.9425
Epoch 8/10000
4/4 [==============================] - 0s 12ms/step - loss: 312.6804 - val_loss: 261.2850
Epoch 9/10000
4/4 [==============================] - 0s 12ms/step - loss: 299.5287 - val_loss: 265.6185
Epoch 10/10000
4/4 [==============================] - 0s 12ms

4/4 [==============================] - 0s 13ms/step - loss: 60.1576 - val_loss: 124.6612
Epoch 80/10000
4/4 [==============================] - 0s 11ms/step - loss: 111.9958 - val_loss: 79.0022
Epoch 81/10000
4/4 [==============================] - 0s 11ms/step - loss: 73.1577 - val_loss: 87.1927
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 88.2854 - val_loss: 55.9796
Epoch 83/10000
4/4 [==============================] - 0s 12ms/step - loss: 57.3921 - val_loss: 61.6995
Epoch 84/10000
4/4 [==============================] - 0s 12ms/step - loss: 53.8777 - val_loss: 64.5934
Epoch 85/10000
4/4 [==============================] - 0s 11ms/step - loss: 57.1401 - val_loss: 53.1419
Epoch 86/10000
4/4 [==============================] - 0s 12ms/step - loss: 44.6053 - val_loss: 69.7863
Epoch 87/10000
4/4 [==============================] - 0s 11ms/step - loss: 52.7952 - val_loss: 77.5927
Epoch 88/10000
4/4 [==============================] - 0s 11ms/step - loss: 69.6442 - v

4/4 [==============================] - 0s 12ms/step - loss: 45.2329 - val_loss: 58.1828
Epoch 159/10000
4/4 [==============================] - 0s 12ms/step - loss: 39.4244 - val_loss: 63.0783
Epoch 160/10000
4/4 [==============================] - 0s 12ms/step - loss: 46.8636 - val_loss: 46.8969
Epoch 161/10000
4/4 [==============================] - 0s 12ms/step - loss: 43.9373 - val_loss: 54.6856
Epoch 162/10000
4/4 [==============================] - 0s 12ms/step - loss: 44.6673 - val_loss: 55.2880
Epoch 163/10000
4/4 [==============================] - 0s 11ms/step - loss: 40.2844 - val_loss: 61.0234
Epoch 164/10000
4/4 [==============================] - 0s 12ms/step - loss: 39.6130 - val_loss: 53.2554
Epoch 165/10000
4/4 [==============================] - 0s 11ms/step - loss: 38.4586 - val_loss: 54.5856
Epoch 166/10000
4/4 [==============================] - 0s 11ms/step - loss: 44.6519 - val_loss: 63.7025
Epoch 167/10000
4/4 [==============================] - 0s 10ms/step - loss: 36.9

Epoch 237/10000
4/4 [==============================] - 0s 11ms/step - loss: 28.1251 - val_loss: 44.1670
Epoch 238/10000
4/4 [==============================] - 0s 11ms/step - loss: 35.1295 - val_loss: 55.6693
Epoch 239/10000
4/4 [==============================] - 0s 14ms/step - loss: 32.1802 - val_loss: 41.4185
Epoch 240/10000
4/4 [==============================] - 0s 12ms/step - loss: 26.1598 - val_loss: 41.6718
Epoch 241/10000
4/4 [==============================] - 0s 11ms/step - loss: 36.6145 - val_loss: 45.5229
Epoch 242/10000
4/4 [==============================] - 0s 11ms/step - loss: 29.8602 - val_loss: 50.9712
Epoch 243/10000
4/4 [==============================] - 0s 11ms/step - loss: 29.5621 - val_loss: 73.6545
Epoch 244/10000
4/4 [==============================] - 0s 12ms/step - loss: 34.2044 - val_loss: 43.8984
Epoch 245/10000
4/4 [==============================] - 0s 13ms/step - loss: 29.7449 - val_loss: 57.0328
Epoch 246/10000
4/4 [==============================] - 0s 12ms/s

Epoch 316/10000
4/4 [==============================] - 0s 14ms/step - loss: 23.0010 - val_loss: 40.9143
Epoch 317/10000
4/4 [==============================] - 0s 12ms/step - loss: 26.1863 - val_loss: 56.4967
Epoch 318/10000
4/4 [==============================] - 0s 12ms/step - loss: 42.4518 - val_loss: 52.1581
Epoch 319/10000
4/4 [==============================] - 0s 12ms/step - loss: 35.2725 - val_loss: 69.2560
Epoch 320/10000
4/4 [==============================] - 0s 12ms/step - loss: 35.5178 - val_loss: 38.0696
Epoch 321/10000
4/4 [==============================] - 0s 12ms/step - loss: 39.8843 - val_loss: 35.2513
Epoch 322/10000
4/4 [==============================] - 0s 12ms/step - loss: 23.0392 - val_loss: 42.4867
Epoch 323/10000
4/4 [==============================] - 0s 11ms/step - loss: 26.6084 - val_loss: 63.7445
Epoch 324/10000
4/4 [==============================] - 0s 11ms/step - loss: 27.9629 - val_loss: 53.2457
Epoch 325/10000
4/4 [==============================] - 0s 11ms/s

...

4/4 [==============================] - 0s 10ms/step - loss: 10.1189 - val_loss: 31.8113
Epoch 1575/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.5293 - val_loss: 32.5311
Epoch 1576/10000
4/4 [==============================] - 0s 11ms/step - loss: 10.1954 - val_loss: 29.0864
Epoch 1577/10000
4/4 [==============================] - 0s 11ms/step - loss: 9.8924 - val_loss: 30.3201
Epoch 1578/10000
4/4 [==============================] - 0s 11ms/step - loss: 9.6561 - val_loss: 28.0917
Epoch 1579/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.9254 - val_loss: 30.4578
Epoch 1580/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.2446 - val_loss: 41.4755
Epoch 1581/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.9326 - val_loss: 30.1818
Epoch 1582/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.9573 - val_loss: 26.6490
Epoch 1583/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.9

Epoch 1653/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.4338 - val_loss: 37.4732
Epoch 1654/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.7636 - val_loss: 33.2174
Epoch 1655/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.8970 - val_loss: 29.9290
Epoch 1656/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.2816 - val_loss: 32.7191
Epoch 1657/10000
4/4 [==============================] - 0s 11ms/step - loss: 8.6443 - val_loss: 32.9244
Epoch 1658/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.3807 - val_loss: 34.4060
Epoch 1659/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.8373 - val_loss: 33.0865
Epoch 1660/10000
4/4 [==============================] - 0s 11ms/step - loss: 10.2712 - val_loss: 29.1875
Epoch 1661/10000
4/4 [==============================] - 0s 10ms/step - loss: 10.4959 - val_loss: 35.4790
Epoch 1662/10000
4/4 [==============================] - 0s 9ms/

Epoch 1732/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.9385 - val_loss: 32.4951
Epoch 1733/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.2477 - val_loss: 32.3568
Epoch 1734/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.5458 - val_loss: 32.0642
Epoch 1735/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.9480 - val_loss: 32.3873
Epoch 1736/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.5453 - val_loss: 34.3097
Epoch 1737/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.9248 - val_loss: 33.9991
Epoch 1738/10000
4/4 [==============================] - 0s 11ms/step - loss: 8.5175 - val_loss: 36.8283
Epoch 1739/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.0384 - val_loss: 39.1435
Epoch 1740/10000
4/4 [==============================] - 0s 11ms/step - loss: 8.9788 - val_loss: 38.1871
Epoch 1741/10000
4/4 [==============================] - 0s 11ms/step

Epoch 1811/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.4669 - val_loss: 36.0796
Epoch 1812/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.4582 - val_loss: 33.6953
Epoch 1813/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.0936 - val_loss: 36.5425
Epoch 1814/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.3768 - val_loss: 30.2043
Epoch 1815/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.2140 - val_loss: 30.6096
Epoch 1816/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.0707 - val_loss: 28.1284
Epoch 1817/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.0085 - val_loss: 29.2197
Epoch 1818/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.0494 - val_loss: 28.9743
Epoch 1819/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.0190 - val_loss: 31.9468
Epoch 1820/10000
4/4 [==============================] - 0s 10ms/step 

4/4 [==============================] - 0s 10ms/step - loss: 6.6638 - val_loss: 35.0674
Epoch 1891/10000
4/4 [==============================] - 0s 11ms/step - loss: 9.8462 - val_loss: 28.7262
Epoch 1892/10000
4/4 [==============================] - 0s 11ms/step - loss: 7.9680 - val_loss: 39.6456
Epoch 1893/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.6949 - val_loss: 35.3996
Epoch 1894/10000
4/4 [==============================] - 0s 12ms/step - loss: 8.0526 - val_loss: 30.0379
Epoch 1895/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.0413 - val_loss: 30.9473
Epoch 1896/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.3420 - val_loss: 35.6657
Epoch 1897/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.1522 - val_loss: 30.1778
Epoch 1898/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.3584 - val_loss: 31.3062
Epoch 1899/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.2405 

In [19]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [20]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,74.67247,74.490952,74.199974,73.963394,73.724922,73.540817,73.53389,73.8349,74.046295,73.95742,73.894699,74.28804,74.140236,74.351776,75.356552,75.401466,74.849419,74.393669,75.01075,76.2686,76.676933,76.703094,76.973862,77.375587,77.884796,78.459778,78.900948,79.27816,80.350021,82.463867
Target,80.517,81.348,76.509,71.851,71.187,58.02,63.194,60.58,69.448,74.472,63.776,81.732,83.858,87.863,69.723,72.765,77.937,64.332,77.048,65.66,65.445,78.095,80.998,69.415,87.346,82.888,79.287,81.855,77.495,56.612
Error,5.844528,6.857048,2.309029,2.112396,2.537926,15.520817,10.33989,13.254898,4.598297,0.51458,10.118698,7.443962,9.717766,13.511223,5.633553,2.636467,3.087578,10.061668,2.037247,10.608597,11.231934,1.391907,4.024139,7.960587,9.461205,4.428223,0.386055,2.576843,2.855019,25.851868


In [21]:
display(mae)
display(mape)

6.963798

0.10136713

In [22]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [23]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined